### SRT Example

In [16]:
from absbox import Generic

seniorBalance = 1000
srtTrancheBal = 200
srtRate = 0.08
closingDate = "2021-06-15"

periodPrincipal = ("curPoolCollection", None, "Principal","Prepayment","Recovery")

pctOfSrt = ("/", ("bondBalance","B")
               , ("sum", ("bondBalance","B"), ("bondBalance","A1")))

pctOfSenior =  ("/", ("bondBalance","A1")
               , ("sum", ("bondBalance","B"), ("bondBalance","A1")))

dueAmountForSrt = ("*", periodPrincipal, pctOfSrt)
dueAmountForSenior = ("*", periodPrincipal, pctOfSenior)


srt01 = Generic(
    "SRT Example"
    ,{"cutoff":"2021-06-01","closing":closingDate,"firstPay":"2021-07-20"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":24
          ,"freq":"Monthly","type":"Level","originDate":"2021-06-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":24
          ,"status":"current"}]],
     }
    ,(("acc01",{"balance":0})
      ,("srtAcc",{"balance":srtTrancheBal})
      ,("dummy",{"balance":0.0})
      ,)
    ,(("A1",{"balance":seniorBalance
             ,"rate":0.00
             ,"originBalance":seniorBalance
             ,"originRate":0.00
             ,"startDate":closingDate
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":srtTrancheBal
             ,"rate":srtRate
             ,"originBalance":srtTrancheBal
             ,"originRate":srtRate
             ,"startDate":closingDate
             ,"rateType":{"Fixed":srtRate}
             ,"bondType":{"Sequential":None}
             }
       ))
    ,tuple()
    ,{"amortizing":[
          # pay prorata to senior and SRT tranch
         ['accrueAndPayInt',"dummy",["A1"],{"support":["facility","originator"]}]
         ,["calcInt","B"]
         ,["payInt","dummy",["B"],{"support":["facility","originator"]}]
         ,["inspect","bond:A1 Bal",("bondBalance","A1","B")]
         ,["If",[("bondBalance","A1","B"),">",0]
               ,["payPrin","acc01",["A1"],{"limit":{"formula":dueAmountForSenior}}]
               ]
         ,["If",[("bondBalance","A1","B"),">",0]
               ,["payPrin","srtAcc",["B"],{"limit":{"formula":dueAmountForSrt}}]
               ]
     ],
      "endOfCollection":[
          ["liqRepay","bal", "srtAcc", "originator", {"formula":("curPoolCollection",None,"Losses")}]
          #["transfer","srtAcc","",{"formula":("cumPoolCollection","Losses")}]
      ],
      "cleanUp":[
          ["payIntResidual","srtAcc","B"]
      ]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,{"originator":{"type" : "Unlimited", "start": closingDate}}
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [17]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL, lang='english', check=False)

r = localAPI.run(srt01
                ,poolAssump=("Pool",("Mortgage", {"CDR":0.01}, None, None, None)
                                   ,None
                                   ,None)
                ,runAssump=[("inspect"#,(["DayOfMonth",20],pctOfSrt)
                                      ,(["DayOfMonth",20],("poolFactor",)))
                            # ,("stop","2023-01-01")
                            ,("call",("poolFactor",0.20))
                           ]
                ,read=True)

Connecting engine server -> http://localhost:8081

AbsboxError: ❌[bold red]Error: Can't not connect to API server http://localhost:8081

In [10]:
r['bonds']['B']

,balance,interest,principal,rate,cash,factor,memo
date,,,,,,,
2021-07-20,200.00,1.53,0.00,0.08,1.53,1.00000,"[<PayInt:B>, <PayPrin:B>]"
2021-08-20,185.60,1.35,14.40,0.08,15.75,0.92800,"[<PayInt:B>, <PayPrin:B>]"
2021-09-20,171.11,1.26,14.49,0.08,15.75,0.85555,"[<PayInt:B>, <PayPrin:B>]"
2021-10-20,156.54,1.12,14.57,0.08,15.69,0.78270,"[<PayInt:B>, <PayPrin:B>]"
2021-11-20,141.88,1.06,14.66,0.08,15.72,0.70940,"[<PayInt:B>, <PayPrin:B>]"
2021-12-20,126.27,0.93,15.61,0.08,16.54,0.63135,"[<PayInt:B>, <PayPrin:B>]"
2022-01-20,110.57,0.85,15.70,0.08,16.55,0.55285,"[<PayInt:B>, <PayPrin:B>]"
2022-02-20,94.78,0.75,15.79,0.08,16.54,0.47390,"[<PayInt:B>, <PayPrin:B>]"
2022-03-20,78.89,0.58,15.89,0.08,16.47,0.39445,"[<PayInt:B>, <PayPrin:B>]"


In [11]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [12]:
r['accounts']['srtAcc']

,balance,change,memo
date,,,
2021-06-15,200.00,0.00,<Support:originator>
2021-06-30,200.00,0.00,<Support:originator>
2021-07-20,200.00,0.00,"[<PayPrin:B>, <DS:Multiply [PoolCurCollection ..."
2021-07-31,198.47,-1.53,<Support:originator>
2021-08-20,184.07,-14.40,"[<PayPrin:B>, <DS:Multiply [PoolCurCollection ..."
2021-08-31,182.72,-1.35,<Support:originator>
2021-09-20,168.23,-14.49,"[<PayPrin:B>, <DS:Multiply [PoolCurCollection ..."
2021-09-30,166.97,-1.26,<Support:originator>
2021-10-20,152.40,-14.57,"[<PayPrin:B>, <DS:Multiply [PoolCurCollection ..."


In [13]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-06-15,2200.00,0.00,0.00,0,0.00,0,0.00,0.08,None,None,0.00,0,0,0.00,0,0.00
2021-06-30,2200.00,0.00,0.00,0,0.00,0,0.00,0.08,None,None,0.00,0,0,0.00,0,0.00
2021-07-31,2113.43,84.76,14.65,0,1.81,0,1.81,0.08,None,None,84.76,0,0,1.81,0,1.81
2021-08-31,2026.37,85.26,14.07,0,1.80,0,1.80,0.08,None,None,170.02,0,0,3.61,0,3.61
2021-09-30,1938.90,85.75,13.49,0,1.72,0,1.72,0.08,None,None,255.77,0,0,5.33,0,5.33
2021-10-31,1851.05,86.25,12.91,0,1.60,0,1.60,0.08,None,None,342.02,0,0,6.93,0,6.93
2021-11-30,1762.72,86.76,12.32,0,1.57,0,1.57,0.08,None,None,428.78,0,0,8.50,0,8.50
2021-12-31,1674.01,87.26,11.74,0,1.45,0,1.45,0.08,None,None,516.04,0,0,9.95,0,9.95
2022-01-31,1584.82,87.77,11.14,0,1.42,0,1.42,0.08,None,None,603.81,0,0,11.37,0,11.37


In [14]:
from absbox.local.util import unifyTs

In [15]:
unifyTs(r['result']['inspect'].values())

,<PoolFactor>
Date,
2021-06-01,0.000000
2021-06-20,1.000000
2021-07-20,1.000000
2021-08-20,0.960650
2021-09-20,0.921077
2021-10-20,0.881318
2021-11-20,0.841386
2021-12-20,0.801236
2022-01-20,0.760913
